<a href="https://colab.research.google.com/github/joshIsac/LargeLanguageModel/blob/main/2348523_LabExcercise2_demo_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#implementation of a descriptive Question answering System
!pip install transformers datasets huggingface_hub
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,TrainingArguments,Trainer

tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset
ds = load_dataset("SoorajK1/questions_and_answers")

Generating train split:   0%|          | 0/29438 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2830 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2830 [00:00<?, ? examples/s]

In [4]:
ds.shape

{'train': (29438, 4), 'validation': (2830, 4), 'test': (2830, 4)}

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["question"], examples["answer"])

tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/29438 [00:00<?, ? examples/s]

Map:   0%|          | 0/2830 [00:00<?, ? examples/s]

Map:   0%|          | 0/2830 [00:00<?, ? examples/s]

In [6]:
#Creation of prompt
def create_prompt(question, context):
  prompt_template = "Answer the following question: {}\n"
  if context:
    prompt_template += "Context: {}\n"
  prompt = prompt_template.format(question, context)
  return prompt

In [7]:
# # def create_prompt_interactive(ds):
#   """
#   This function allows users to interactively create prompts for question answering.
#   """
#   while True:
#     # Get user input for a question or 'quit' to exit
#     question = input("Enter your question (or 'quit' to exit): ")
#     if question.lower() == 'quit':
#       break

#     # Iterate over datasets within the DatasetDict
#     for dataset_name in ds.keys():
#       dataset = ds[dataset_name]

#       # Find the most relevant question based on word overlap
#       best_match_question = None
#       best_match_count = 0
#       for q in dataset['question']:
#         count = sum(1 for word in question.lower().split() if word in q.lower().split())
#         if count > best_match_count:
#           best_match_count = count
#           best_match_question = q

#       if best_match_question:
#         # Find the index of the best matching question
#         match_index = dataset['question'].index(best_match_question)
#         context = dataset[match_index]['answer']  # Use the index to get the context


#         # Create the prompt (call a separate function not shown here)
#         prompt = create_prompt(question, context)
#         print("Generated Prompt:\n", prompt)

#         # Exit the outer loop if a relevant question is found
#         break

#     else:  # This else is associated with the 'for' loop, not the 'if'
#       print("I'm unaware of this question. Please try another one.")

In [8]:
def create_prompt_interactive(ds):
  """
  This function allows users to interactively create prompts for question answering.
  """
  while True:
    # Get user input for a question or 'quit' to exit
    question = input("Enter your question (or 'quit' to exit): ")
    if question.lower() == 'quit':
      break

    # Iterate over datasets within the DatasetDict
    relevant_question_found = False  # Flag to track if a relevant question is found
    for dataset_name in ds.keys():
      dataset = ds[dataset_name]

      # Find the most relevant question based on word overlap
      best_match_question = None
      best_match_count = 0
      for q in dataset['question']:
        count = sum(1 for word in question.lower().split() if word in q.lower().split())
        if count > best_match_count:
          best_match_count = count
          best_match_question = q

      if best_match_question:
        # Find the index of the best matching question
        match_index = dataset['question'].index(best_match_question)
        context = dataset[match_index]['answer']  # Use the index to get the context

        # Create the prompt (call a separate function not shown here)
        prompt = create_prompt(question, context)
        print("Generated Prompt:\n", prompt)

        relevant_question_found = True  # Set the flag to True
        break  # Exit the inner loop if a relevant question is found

    if not relevant_question_found:  # Check the flag after the loop
      print("I'm unable to answer this question. Please try another one.")


In [9]:
create_prompt_interactive(ds)

Enter your question (or 'quit' to exit): What data needs to be provided by the Insurance Company?
Generated Prompt:
 Answer the following question: What data needs to be provided by the Insurance Company?
Context: The Insurance Company needs to provide certain data as per the requirement.

Enter your question (or 'quit' to exit): Who may chair the STAC?
Generated Prompt:
 Answer the following question: Who may chair the STAC?
Context: The STAC may be chaired by the Principal Secretary/Secretary of the concerned department.

Enter your question (or 'quit' to exit): What is the role of TAC/TA/STAC?
Generated Prompt:
 Answer the following question: What is the role of TAC/TA/STAC?
Context: TAC/TA/STAC deliberates and examines the subject matter and indicates their views/decisions based on statistical/technological inputs and other related scientific and technical parameters as detailed in the SOP.

Enter your question (or 'quit' to exit): quit


in this Descriptive Q and A we have loaded the datasets  and based on the datasets  a fuction is created for entering the user prompt and the answer corresponding to the question is generated